In [1]:
#### For GOOGLE COLAB and similar platform Users:
#### Make sure to select a GPU in the online platform. Don't run this code with a CPU (it will be too slow)

# If you are running this code locally, your GPU should be selected automatically

In [2]:
# Run this cell only if you havent installed these libraries already outside of the notebook
#!pip install -q ipdb
#!pip install -q transformers

# And if you are not in Google Colab and you didn't yet install Pytorch, make sure to do it:
# find the ideal pytorch installation command at https://pytorch.org/get-started/locally/

# Official Notebook #vj30

In [3]:
# You can use this command to view information about your GPU and the amount of free memory it has
# Make sure that you have at last 4GB of free GPU memory to do this course
!nvidia-smi 
# If you are using Google Colab or a similar online platform, make sure to select a GPU in the menus
# In Google colab, at the moment the option is within the Runtime menus

Fri Feb 21 08:37:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.35                 Driver Version: 556.35         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A2000 8GB Lap...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   44C    P8              4W /   49W |    2383MiB /   8192MiB |     38%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# If you are running this online (for example at Google Colab), 
# make sure you have the support files on the same folder
# Otherwise run this cell to download them

# NOTE: Downloading will take a while, be patient. You can refresh your folder from time to time to see when the files
# have been created.

import os, requests, zipfile, io 

files_url = "https://ideami.com/llm_align"

# Downloading proceeds if we detect that one of the key files to download is not present
if not os.path.exists(f"llm.py"):
    print("Downloading files using Python")
    response = requests.get(files_url)
    zipfile.ZipFile(io.BytesIO(response.content)).extractall(".")
else:
    print("you seem to have already downloaded the files. If you wish to re-download them, delete the llm.py file")


you seem to have already downloaded the files. If you wish to re-download them, delete the llm.py file


In [5]:
import os
# Pytorch
import torch
# Architecture
import transformers
# Import Llama based model
from llm import Llama, ModelArgs

In [6]:
use_orpo = True  # use aligned checkpoint or not
num_answers = 1
temp = 0.5
topk= 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("You use device: ", device)
    
tokenizer_path = "tokenizers/tok16384"
model_path = "./models/"
    
if use_orpo==True:
        model_inf, context= "newModelLLama_3.pt", 1024  # ORPO is trained with context of 1024
        print("Mode::Using Orpo aligned model")
else:
        model_inf, context= "base_model.pt", 512  # The original was trained with context of 512
        print("Mode::Using pretrained model without alignment")

print(f"Using model {model_inf}")
   
# Load model and extract config
checkpoint = torch.load(os.path.join(model_path, model_inf), map_location=device)
config = checkpoint.pop("config")
    
# temporary fix if the model was trained and saved with torch.compile
# The _orig_mod. prefix in your model's state dictionary keys is related to
# how PyTorch handles compiled models, specifically when using the torch.compile function
# When torch.compile is used, PyTorch might wrap the original model in a way that modifies
# the names of its parameters and buffers. This wrapping can prepend a prefix like _orig_mod.
# We remove those wrappings to make the checkpoint compatible with the non compiled version of the model
new_dict = dict()
for k in checkpoint.keys():
        if k.startswith("_orig_mod."):
            #print("Removing _orig_mod wrapping")
            new_dict[k.replace("_orig_mod.", "")] = checkpoint[k]
        else:
            new_dict[k] = checkpoint[k]





You use device:  cuda
Mode::Using Orpo aligned model
Using model newModelLLama_3.pt


In [8]:
# Setup tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_path)
tokenizer.pad_token = tokenizer.eos_token

diviseurPerf = 4
model_args = ModelArgs(
    dim = 4096//diviseurPerf, 
    n_layers = 32//diviseurPerf,  
    n_heads = 32//diviseurPerf, 
    n_kv_heads =  8, 
    vocab_size = 128256//diviseurPerf, 
    multiple_of = 256,  
    ffn_dim_multiplier = None,
    norm_eps = 1e-06, 
    rope_theta = 500000//diviseurPerf, 
    max_seq_len = 8192//diviseurPerf, 
    dropout = 0.1, 
    hidden_dim = 14336//diviseurPerf,
    attention_bias = True,
    mlp_bias = True, 
)


# Instantiate model, load parms, move to device
model = Llama(model_args)
model.load_state_dict(new_dict)
if device.type == 'cuda':
        model = model.to(torch.bfloat16)
        model = model.to(device)
model.eval()

model_size = sum(t.numel() for t in model.parameters())
print(f"Model size: {model_size/1e6:.2f} M parameters")

Model size: 187.42 M parameters


In [9]:
tokens = tokenizer.encode("Test input")
print(f"Encoded tokens: {tokens}")
print(f"Vocabulary size: {model.vocab_size}")



Encoded tokens: [5435, 5567]
Vocabulary size: 32064


In [10]:

# Interactive loop
while True:
    qs = input("Enter text (q to quit) >>> ")
    if qs == "":
        continue
    if qs == 'q':
        break

    # we activate chat template only for ORPO model because it was trained with it
    if use_orpo:
        qs = f"<s> <|user|>\n{qs}</s>\n<s> <|assistant|> "

    x = tokenizer.encode(qs)
    x = torch.tensor(x, dtype=torch.long, device=device)[None, ...]
    
    print(f"Model vocabulary size: {tokenizer.vocab_size}")
    print(f"Max token in input: {x.max()}")


    for ans in range(num_answers):
        with torch.no_grad():
            y = model.generate(
                x, 
                max_new_tokens=256, 
                temperature=temp, 
                top_k=topk
            )

        response = tokenizer.decode(y[0].tolist(), skip_special_tokens=True)   

        output = model.clean_response(response)

        print("################## \n")
        print(f"### Answer {ans+1}: {output}")

Model vocabulary size: 16384
Max token in input: 9948
################## 

### Answer 1: 
hi 
  

To find the sum of the sum of the sum of the sum of a sum of the sum of the sum of the sum, I can use the sum of the sum of the sum of the same as the sum of the sum of the sum of the sum is the sum of the sum of the sum of the sum of the sum of the sum is the sum of the sum of the sum of the sum of the sum is 2 times the sum of the sum of the sum of the sum is 5 + 2 = 2 + 2 + 2 + 3 = 1 + 2 + 2 + 3 + 3 + 2 + 2 = 10 + 2 + 2 + 2 + 2 + 3 + 2 + 3 + 2 + 3 + 3 + 6 + 2 + 2 + 2 + 2 + 2 = 3 + 10 + 3 + 1 + 2 + 5 = 3 + 1 = 5 + 5 + 3 = 3 + 3 + 3 + 2 + 3 = 16 + 2 + 4 + 2 + 5 = 3 + 3 + 2 + 3 + sides, I can plug in the sum of an integers in the sum of the sum is 2 + 2 + 2 + 3 = 2 + 3 + 2 + 2 + 2 + 2 + 3 + 2 = 2 +
Model vocabulary size: 16384
Max token in input: 9948
################## 

### Answer 1: 
Give me an text from your mind 
   <</(->

Then, I'd be happy to help you for your product to use the fo